## Generate Subjective Value (SV) difference

Use the BHM generated parameters to generate SV difference


### Import libraries and modules

In [19]:
import pandas as pd
import os,sys
pd.options.display.max_rows = 999
pd.options.display.max_columns = 99

In [20]:
# getting the name of the directory
# where the this file is present.
# current = os.path.dirname(os.path.realpath(__file__))
current = os.path.dirname(os.getcwd())
 
# Getting the parent directory name
# where the current directory is present.
# parent = os.path.dirname(os.path.dirname(current))
# parent = current
parent = '/Users/pizarror/IDM'
# print(parent)
#/Users/pizarror/IDM

# adding the parent directory to
# the sys.path.
sys.path.append(parent)

from IDM_model.src import model_functions as mf


In [21]:
def get_df_bhm(idm_dir=''):
    agg_fn = os.path.join(idm_dir,'Aggregate_All_Model_Parameters.csv')
    df_agg = pd.read_csv(agg_fn)
    cols = [c for c in list(df_agg) if 'bhm' in c or 'subject' in c]
    df_bhm = df_agg.loc[:,cols]
    return df_bhm


def read_load_data(subject='23_IDM_0144',fn='/tmp',alpha0=1.0,cols=[]):
    crdm_df = pd.read_csv(fn)
    task='crdm'
    crdm_df = mf.drop_pract(crdm_df,task=task)
    crdm_df,response_rate = mf.drop_non_responses(crdm_df,task=task,verbose=False) 
    data = mf.get_data(crdm_df,cols,task='crdm')[0]
    data['subject'] = subject
    return data 

In [22]:
idm_dir = '/Volumes/UCDN/datasets/IDM/'
df_bhm = get_df_bhm(idm_dir=idm_dir)

split_dir = os.path.join(idm_dir,'split')

task = 'crdm'
cols = ['crdm_choice','crdm_sure_p','crdm_lott_p','crdm_sure_amt','crdm_lott_amt','crdm_amb_lev']
data = pd.DataFrame(columns=['subject']+cols)

for s in df_bhm['subject']:
    sfn = os.path.join(split_dir,s,task,'{}_{}.csv'.format(s,task))
    subj_data = read_load_data(subject=s,fn=sfn,alpha0=1.0,cols=cols)
    for c in cols:
        subj_data[c] = subj_data[c].astype(float)        
    data = pd.concat([data,subj_data],ignore_index=True)

display(data)

,subject,crdm_choice,crdm_sure_p,crdm_lott_p,crdm_sure_amt,crdm_lott_amt,crdm_amb_lev
0,23_IDM_0001,1.0,1.0,0.50,5.0,20.0,0.50
1,23_IDM_0001,1.0,1.0,0.25,5.0,50.0,0.00
2,23_IDM_0001,0.0,1.0,0.75,5.0,5.0,0.00
3,23_IDM_0001,0.0,1.0,0.25,5.0,5.0,0.00
4,23_IDM_0001,0.0,1.0,0.13,5.0,5.0,0.00
...,...,...,...,...,...,...,...
11799,23_IDM_0149,0.0,1.0,0.13,5.0,20.0,0.00
11800,23_IDM_0149,1.0,1.0,0.50,5.0,50.0,0.50
11801,23_IDM_0149,0.0,1.0,0.38,5.0,50.0,0.00
11802,23_IDM_0149,1.0,1.0,0.50,5.0,50.0,0.74
